## This notebook shows how to run a whole lot of propagations

In [ ]:
import adam
from adam import Batch
from adam import PropagationParams
from adam import OpmParams
from adam import BatchRunManager
from adam import ConfigManager
from adam import Service

import datetime
import os
import pandas as pd

This sets up a Service which uses the given token and URL to provide authorized access through the server through several wrapped modules. It also creates a project for you to work in that will be used for the rest of the notebook. Be sure to run service.teardown() when finished. 

If you don't have a config, see config_demo to get one.

In [ ]:
cm = adam.ConfigManager()
service = adam.Service.from_config(cm.get_config())
service.setup()

In [ ]:
working_project = service.new_working_project()

This function creates a propagation object. For now, it just creates a dummy object with slight variations to avoid creating duplicate objects. For detailed information on Batch creation, see test_single_batch_run notebook.

In [ ]:
def get_batch(i):
    state_vec = [130347560.13690618,
                 -74407287.6018632,
                 -35247598.541470632,
                 23.935241263310683,
                 27.146279819258538,
                 10.346605942591514]
    
    start_time = datetime.datetime(2018, 2, 21, 0, 0, 0, 123456 + i)
    end_time = datetime.datetime(2028, 2, 21, 0, 0, 0, 123456 + i)  # Ten years later.
    
    return Batch(PropagationParams({
        'start_time': start_time.isoformat() + 'Z',
        'end_time': end_time.isoformat() + 'Z',
        'project_uuid': working_project.get_uuid(),
    }), OpmParams({
        'epoch': start_time.isoformat() + 'Z',
        'state_vector': state_vec,
        'mass': 500 + i,  # Create a small variation between batches.
    }))

This creates a large number of batch runs and propagates them. This'll probably take ~10 minutes. It'll be faster to run the second time because the server is already all scaled up.

In [ ]:
num_batches = 1000
batches = [get_batch(i) for i in range(num_batches)]

batch_run_manager = BatchRunManager(service.get_batches_module(), batches)
batch_run_manager.run()

TODO(laura): This is actually not true because python notebooks do not run cells simultaneously. There are ways to make this work. Do it.

To view the status of your runs while they're propagating, run this.

In [ ]:
status = batch_run_manager.get_latest_statuses()
status_counts = [[s, len(status[s])] for s in status]
df = pd.DataFrame(status_counts, columns=['Task', 'Status'])
df.style.hide_index()

Once the batches are all done propagating, you can view their results.

In [ ]:
for batch in batches:
    if batch.get_calc_state() == 'FAILED':
        error = 'unknown error'
        if batch.get_results() is not None and batch.get_results().get_parts()[0] is not None:
            error = batch.get_results().get_parts()[0].get_error()
        #print('Batch %s failed: %s' % (batch.get_uuid(), error))
    else:
        final_state_vector = batch.get_results().get_end_state_vector()
        #print('Batch %s ended at %s' % (batch.get_uuid(), final_state_vector))#

In [ ]:
batch_dict = { batch.get_uuid() : batch.get_results().get_end_state_vector() for batch in batches }
df = pd.DataFrame.from_dict(batch_dict,orient='index', columns=['Rx','Ry','Rz', 'Vx', 'Vy', 'Vz'])
df.style.set_caption('Batch UUID vs. Final State Vector')

That's it! Please clean up by running the following:


In [ ]:
batch_dict = { batch.get_uuid() : batch.get_results().get_parts()[0].get_error() for batch in batches }
df = pd.DataFrame.from_dict(batch_dict,orient='index', columns=['Error'])
df.style.set_caption('Batch UUID vs. Error')

In [ ]:
service.teardown()